In [23]:
import requests
import json

In [24]:
access_token = !gcloud auth application-default print-access-token
access_token = '\n'.join(access_token)

project = 'boreal-ward-437408-p6'
base_url = "https://generativelanguage.googleapis.com"

In [25]:
headers={
  'Authorization': 'Bearer ' + access_token,
  'Content-Type': 'application/json',
  'x-goog-user-project': project
}

result = requests.get(
  url=f'{base_url}/v1beta/tunedModels',
  headers = headers,
)

In [ ]:
import time
from bert_score import score
import numpy as np

test_cases = [
    {
        "query": "What is atelectasis?",
        "expected": "Atelectasis is the collapse of a lung or part of a lung, also known as a lobe. It occurs when the tiny air sacs in the lung, called alveoli, lose air. This condition can impede proper gas exchange, making breathing difficult."
    },
    {
        "query": "How is atelectasis diagnosed?",
        "expected": "Atelectasis is typically diagnosed using imaging tests such as chest X-rays or CT scans. Pulmonary function tests may also be used to assess lung function. Accurate diagnosis is essential to determine the appropriate treatment and rule out other conditions."
    },
    {
        "query": " How is cardiomegaly (enlarged heart) diagnosed?",
        "expected": "To diagnose an enlarged heart, a health care provider typically begins with a physical exam and asks about your symptoms and medical history. Common diagnostic tests include blood tests to rule out conditions like heart attacks, chest X-rays to check the size of the heart, and electrocardiograms (ECGs) to assess heart rhythms. An echocardiogram may be done to get a detailed view of the heart's size and function using sound waves. In some cases, advanced imaging like cardiac CT or MRI is used, or cardiac catheterization may be performed to examine the heart's internal structures and blood flow."
    },
    {
        "query": "How can you prevent an enlarged heart?",
        "expected": "Adopting a heart-healthy lifestyle can reduce the risk. This includes managing high blood pressure, cholesterol, and diabetes through diet, exercise, and medications as prescribed by a doctor. Avoiding smoking, limiting alcohol, and following a balanced diet rich in fruits, vegetables, and whole grains can also help prevent conditions that lead to cardiomegaly. Regular checkups with a healthcare provider are important for early detection and prevention."
    },
    {
        "query": " What are the symptoms of a pleural effusion?",
        "expected": "The symptoms of a pleural effusion can vary depending on the size of the effusion and whether there is accompanying inflammation. Small effusions may be asymptomatic, while larger ones often cause shortness of breath, chest pain (especially when taking a deep breath), and cough. If the effusion is associated with an infection, fever and general malaise might also be present. Severe effusions can compress the lungs to the point where breathing becomes significantly impaired, requiring urgent medical intervention."
    },
    {
        "query": "What are the risks associated with untreated pleural effusions?",
        "expected": "If left untreated, pleural effusions can lead to serious complications depending on their size and cause. Large effusions can compress the lungs, reducing their capacity and making it difficult to breathe, potentially leading to respiratory failure. Infected effusions can turn into empyema, where pus accumulates in the pleural space, requiring more complex treatment. Chronic effusions can also cause the pleura to thicken and scar, which might result in permanent lung function impairment and complications such as trapped lung."
    },
    {
        "query": "What are the types of lung cancer?",
        "expected": "Lung cancer is classified into two main types: small cell lung cancer (SCLC) and non-small cell lung cancer (NSCLC). SCLC is less common and is almost exclusively found in heavy smokers, while NSCLC is more common and includes subtypes like squamous cell carcinoma, adenocarcinoma, and large cell carcinoma. The type of lung cancer influences the treatment options and prognosis. Treatment plans are tailored based on whether the cancer is small cell or non-small cell and the stage at diagnosis."
    },
    {
        "query": "How is lung cancer treated?",
        "expected": "Treatment for lung cancer depends on the type, stage, and overall health of the patient. Options include surgery, where part or all of the lung may be removed, chemotherapy, radiation therapy, targeted therapy, and immunotherapy. For early-stage cancers, surgery is often the primary treatment, while chemotherapy and radiation are used to shrink tumors or eliminate remaining cancer cells. In advanced cases, treatments aim to control symptoms and improve quality of life rather than cure the disease."
    },
    {
        "query": "How are lung nodules found?",
        "expected": "Lung nodules are typically discovered through imaging tests conducted for various reasons. They can be identified during routine screenings with low-dose CT scans, especially in patients at high risk for lung cancer, such as heavy smokers. Additionally, nodules may be detected incidentally during imaging tests performed for unrelated issues, like after a car accident. Sometimes, lung nodules are found when physicians are investigating symptoms like persistent cough or shortness of breath."
    },
    {
        "query": "Are all lung nodules cancerous?",
        "expected": "No, the majority of lung nodules are benign and not cancerous. Approximately 95% of lung nodules do not indicate lung cancer and may be caused by infections, scar tissue, or other non-cancerous conditions. However, certain characteristics of nodules, such as size, shape, and growth rate, can suggest a higher likelihood of cancer. For instance, nodules that are larger, have irregular or spiculated borders, or show rapid growth over time are more concerning for malignancy. It is essential to evaluate each nodule individually based on the patient’s risk factors and imaging findings."
    },
    {
        "query": "How is pneumonia classified?",
        "expected": "Pneumonia is classified based on the type of pathogen causing the infection and the setting in which it was acquired. The main categories include:  Community-Acquired Pneumonia (CAP): Occurs outside of hospitals or other healthcare facilities. Hospital-Acquired Pneumonia (HAP): Develops during a hospital stay for another illness. Healthcare-Associated Pneumonia (HCAP): Occurs in long-term care facilities or outpatient clinics. Ventilator-Associated Pneumonia (VAP): Develops in people using breathing machines in the hospital. Aspiration Pneumonia: Results from inhaling food, drink, vomit, or saliva into the lungs. Each classification helps determine the appropriate treatment and management strategies."
    },
    {
        "query": "When is hospitalization necessary for pneumonia?",
        "expected": "Hospitalization for pneumonia may be necessary if you:  Are older than 65 or younger than 2 years Have a weakened immune system Have severe symptoms, such as difficulty breathing, high fever, confusion, or chest pain Are unable to take oral medications due to vomiting or severe coughing Have chronic health conditions like heart disease or chronic obstructive pulmonary disease (COPD) Require oxygen therapy or breathing assistance Hospitalization allows for closer monitoring and more intensive treatment, which is crucial for high-risk individuals or those with severe pneumonia."
    },
    {
        "query": " What are the common causes of pneumothorax?",
        "expected": "Pneumothorax can be caused by several factors, including:  Chest Injury: Blunt or penetrating injuries to the chest, such as those from car accidents, physical assaults, or medical procedures involving needle insertion. Lung Disease: Conditions like chronic obstructive pulmonary disease (COPD), cystic fibrosis, lung cancer, and pneumonia can weaken lung tissue, making it more susceptible to collapse. Ruptured Air Blisters (Blebs): Small air sacs on the lung surface can rupture, allowing air to escape into the chest cavity. Mechanical Ventilation: Use of ventilators in intensive care units can create imbalances in air pressure, leading to lung collapse. Spontaneous Causes: Occurs without any obvious cause, often in tall, thin individuals or smokers. Additionally, genetic factors and previous occurrences of pneumothorax can increase the risk of future episodes."
    },
    {
        "query": "What are the treatment options for pneumothorax?",
        "expected": "Treatment for pneumothorax aims to remove the excess air from the chest cavity and allow the lung to re-expand. The method chosen depends on the size and cause of the pneumothorax, as well as the patient's overall health:  Observation: For small pneumothoraxes, especially if asymptomatic, doctors may monitor the condition with regular chest X-rays to ensure the lung re-inflates naturally over several weeks. Needle Aspiration: Involves inserting a needle connected to a syringe between the ribs to remove excess air. This procedure is often used for larger or symptomatic pneumothoraxes and may require leaving a catheter in place temporarily. Chest Tube Insertion: A chest tube is placed between the ribs to continuously remove air until the lung re-expands fully. This method is typically used for more severe pneumothoraxes and may necessitate hospitalization. Nonsurgical Repair: If air continues to leak, treatments like pleurodesis (using a substance to irritate and seal the pleural layers) or autologous blood patch (injecting blood to form a scar and seal leaks) may be employed. Surgery: In recurrent or persistent cases, surgical intervention may be required to repair the lung and prevent future collapses. This can be done using minimally invasive techniques with small incisions and a fiber-optic camera or through more extensive surgery if necessary. Supplemental Oxygen Therapy: Administered to speed up the reabsorption of air and help the lung re-expand more quickly. Post-treatment, patients are advised to avoid activities that increase chest pressure, such as flying or scuba diving, until fully healed."
    },
    {
        "query": "What are the common symptoms of lung consolidation?",
        "expected": "The primary symptom of lung consolidation is difficulty breathing, as the consolidated areas prevent air from entering the lungs effectively. This can lead to shortness of breath and may cause the skin to appear pale or bluish due to insufficient oxygen levels. Other symptoms may include coughing up thick green or bloody sputum, a dry cough, noisy or abnormal breathing sounds, chest pain or heaviness, rapid breathing, fever, and fatigue. The severity of these symptoms often depends on the extent of the lung consolidation."
    },
    {
        "query": "How is lung consolidation different from a pleural effusion?",
        "expected": "While both lung consolidation and pleural effusion appear as white or opaque areas on a chest X-ray, they differ in location and behavior:  Lung Consolidation: Occurs within the lung tissue itself, where air in the alveoli is replaced by fluid, pus, blood, or other material. Consolidated areas do not move when the patient changes position because the material is fixed within the lung.  Pleural Effusion: Involves the accumulation of fluid in the space between the chest wall and the lungs (pleural space). This fluid can move or shift with changes in the patient's position due to its location in an open space.  Additionally, some underlying causes such as pneumonia and lung cancer can lead to both conditions simultaneously, requiring comprehensive diagnostic and treatment approaches."
    },
    {
        "query": "What are the primary symptoms of pulmonary edema?",
        "expected": "The primary symptoms of pulmonary edema include difficulty breathing or extreme shortness of breath, which may worsen with physical activity or when lying down. Individuals may experience a feeling of suffocating or drowning, accompanied by a cough that produces frothy sputum, which can sometimes contain blood. Other common symptoms are rapid and irregular heartbeat (palpitations), anxiety or restlessness, cold and clammy skin, wheezing or gasping for breath, and chest pain or heaviness. In severe cases, pulmonary edema can lead to bluish skin due to a lack of oxygen, confusion, and a significant drop in blood pressure, potentially causing lightheadedness or dizziness."
    },
    {
        "query": "What are the risk factors for developing pulmonary edema?",
        "expected": "Several factors can increase the risk of developing pulmonary edema, including:  Heart Conditions: Chronic heart diseases such as coronary artery disease, heart failure, cardiomyopathy, and heart valve disorders significantly elevate the risk. High Blood Pressure (Hypertension): Uncontrolled high blood pressure can strain the heart and lead to fluid buildup in the lungs. Smoking: Tobacco smoke damages the lungs and heart, making fluid accumulation more probable. Alcohol Use: Excessive alcohol consumption can weaken the heart muscle and contribute to heart failure. High-Altitude Exposure: Traveling to or exercising at high elevations without proper acclimatization can cause high-altitude pulmonary edema (HAPE). Certain Medications: Some drugs, including certain chemotherapy agents and medications for high blood pressure, can increase the risk. Genetic Factors: A family history of heart disease or pulmonary conditions may predispose individuals to pulmonary edema. Also, liver, kidney, lung injury or disease can be risk factors for edema. Managing these risk factors through lifestyle changes and medical interventions can help reduce the likelihood of developing pulmonary edema."
    },
    {
        "query": "What are the complications associated with pulmonary fibrosis?",
        "expected": "Pulmonary fibrosis can lead to several serious complications, including:  Pulmonary hypertension: Elevated blood pressure in the pulmonary arteries, increasing the workload on the heart’s right ventricle. Right-sided heart failure (Cor Pulmonale): Occurs when the right side of the heart struggles to pump blood through the stiffened lungs, leading to heart muscle enlargement and weakening. Respiratory failure: A critical condition where the lungs cannot adequately exchange oxygen and carbon dioxide, requiring emergency medical intervention. Lung cancer: Chronic scarring and inflammation in the lungs elevate the risk of developing lung cancer. Other lung issues: Such as blood clots in the lungs (pulmonary embolism), collapsed lung (pneumothorax), and increased vulnerability to lung infections. Addressing these complications promptly is vital for improving outcomes and maintaining quality of life for individuals with pulmonary fibrosis."
    },
    {
        "query": "What are the risk factors for developing pulmonary fibrosis?",
        "expected": "Risk factors for pulmonary fibrosis include: Smoking: Both current and former smokers have an increased risk. Age: Idiopathic pulmonary fibrosis is more common in middle-aged and older adults. Occupational exposure: Jobs involving continuous or repeated exposure to harmful dusts, chemicals, or pollutants, such as mining, farming, or construction. Genetics: A family history of pulmonary fibrosis can increase risk. Radiation therapy: Receiving radiation treatments to the chest area. Certain medical conditions: Having autoimmune diseases or other lung conditions like emphysema. These factors contribute to the likelihood of developing pulmonary fibrosis, either independently or in combination."
    },
    {
        "query": " How does emphysema impact daily life and activities?",
        "expected": "Emphysema significantly affects daily life and activities by:  Limiting physical activities: Even simple tasks like walking or climbing stairs can cause breathlessness. Reducing exercise capacity: Patients often experience fatigue and may avoid activities that exacerbate their symptoms. Affecting mental health: The chronic nature of the disease can lead to feelings of frustration, anxiety, and depression. Interfering with sleep: Shortness of breath may worsen at night, disrupting sleep patterns. Increasing reliance on others: Severe emphysema can limit independence, requiring assistance with daily tasks. Managing these impacts involves comprehensive treatment plans, including pulmonary rehabilitation and support groups, to improve quality of life."
    },
    {
        "query": "How is emphysema diagnosed?",
        "expected": "Doctors use various methods to diagnose emphysema, which may include:  Medical history and physical exam: Checking for symptoms like shortness of breath, cough, and wheezing. Imaging tests: Chest X-rays or CT scans to detect lung damage and assess its severity. Lung function tests: Spirometry, which measures how well the lungs work by assessing airflow and volume. Blood tests: Arterial blood gases to measure oxygen and carbon dioxide levels in the blood. These methods help physicians confirm the presence of emphysema and rule out other conditions."
    },
    {
        "query": "How is a pulmonary hernia diagnosed, and what imaging techniques are most effective?",
        "expected": "Diagnosing a pulmonary hernia involves a combination of clinical evaluation and imaging studies. The following steps are typically taken:  Clinical Examination: A physical examination may reveal a visible or palpable bulge in the chest wall, especially during maneuvers that increase intra-thoracic pressure like coughing or sneezing. Subcutaneous emphysema may also be detected.  Chest X-Ray: This is often the first imaging modality used. It can reveal abnormalities such as rib fractures, chest wall defects, or the presence of lung tissue outside the normal thoracic cavity.  Computed Tomography (CT) Scan: A CT scan is the most effective imaging technique for diagnosing pulmonary hernias. It provides detailed images that can accurately assess the location, size, and extent of the herniation. CT scans are particularly useful in identifying the exact nature of the chest wall defect and any associated complications, such as pleural effusion.  Functional Respiratory Tests: These may be conducted to evaluate the impact of the hernia on lung function, although they are supplementary to imaging studies.  In some cases, additional imaging like MRI may be utilized, but CT scans remain the gold standard for detailed assessment."
    },
    {
        "query": "What are the primary causes of pulmonary hernias, and what risk factors increase the likelihood of developing this condition?",
        "expected": "Pulmonary hernias are primarily caused by:  Trauma: Severe injuries to the chest area, such as those from traffic accidents, falls, or penetrating injuries, can create openings in the chest wall through which lung tissue may protrude. Surgical Interventions: Thoracic surgeries, especially those involving inadequate closure of the chest wall, can lead to weaknesses that predispose patients to lung herniation. Increased Intra-Thoracic Pressure: Conditions like morbid obesity or chronic obstructive pulmonary disease (COPD) can elevate intra-thoracic pressure, making herniation more likely. Chronic Coughing: Persistent or severe coughing can strain chest muscles and contribute to the formation of hernias. Congenital Factors: Although rare, some individuals are born with structural weaknesses in the chest wall that can lead to pulmonary hernias. Risk factors include:  Obesity, which increases intra-thoracic pressure. History of thoracic surgery or trauma. Chronic respiratory conditions that cause frequent coughing. Tissue weakness due to malnutrition, steroid use, diabetes, or other comorbidities. Anatomical weaknesses in the intercostal muscles or fascia. These factors can either directly cause the hernia or create an environment where a hernia is more likely to develop."
    },
    {
        "query": "What lifestyle changes can help manage pleural thickening?",
        "expected": "Implementing certain lifestyle modifications can significantly aid in managing pleural thickening and improving overall well-being:  Smoking Cessation:  Quitting smoking is crucial, especially for individuals whose pleural thickening is related to asbestos exposure or other respiratory irritants. Reduces further lung damage and improves the effectiveness of treatments. Avoiding Respiratory Irritants:  Minimize exposure to pollutants, dust, chemicals, and other harmful substances. Use protective gear such as masks in environments with high exposure risks. Regular Exercise:  Engage in tailored exercise programs, such as those offered in pulmonary rehabilitation, to enhance muscle strength and lung capacity. Helps improve endurance and reduce fatigue. Healthy Diet:  Maintain a balanced diet to support overall health and immune function. Proper nutrition can help manage weight and energy levels. Breathing Techniques:  Practice diaphragmatic and pursed-lip breathing to improve ventilation efficiency and reduce shortness of breath. Techniques learned in pulmonary rehabilitation can be particularly beneficial. Stress Management:  Incorporate stress-reduction practices like meditation, yoga, or counseling to cope with the emotional challenges of chronic illness. Regular Medical Follow-ups:  Keep up with scheduled appointments and adhere to treatment plans. Early detection of any worsening symptoms allows for timely intervention. These lifestyle changes, combined with medical treatments, can enhance symptom management and improve the quality of life for individuals with pleural thickening."
    },
    {
        "query": "How is pleural thickening diagnosed?",
        "expected": "Diagnosing pleural thickening involves a combination of clinical evaluation and imaging studies:  Medical History and Physical Examination:  Assessing symptoms such as chest pain, cough, and difficulty breathing. Reviewing exposure to risk factors like asbestos or prior chest injuries. Imaging Tests:  Chest X-ray: Can reveal areas of thickened pleura, especially when viewed in profile. Computed Tomography (CT) Scan: High-resolution CT (HRCT) is the primary modality for detailed assessment. Helps determine the extent and location of pleural thickening. Magnetic Resonance Imaging (MRI): Used in select cases for better soft tissue contrast. Positron Emission Tomography (PET) Scan: Assists in differentiating benign from malignant causes by detecting metabolic activity. Ultrasound: Limited role but useful for guiding pleural procedures. Biopsy:  In cases where malignancy is suspected, a tissue sample may be taken for histopathological examination. Early and accurate diagnosis facilitates timely management and improves patient outcomes."
    },
    {
        "query": "How does a pulmonary infiltrate appear on a chest X-ray versus a CT scan?",
        "expected": "Pulmonary infiltrates can be visualized using both chest X-rays and CT scans, but each imaging modality provides different levels of detail and information:  **Chest X-Ray:**  - **Appearance:** On a chest X-ray, pulmonary infiltrates typically appear as hazy or opaque areas within the lung fields. They may present as patchy opacities, consolidation (where the alveoli are filled with fluid, pus, or other substances), or interstitial patterns (indicating thickening of the lung interstitium).  - **Limitations:** While chest X-rays are useful for initial evaluation, they may not detect small or subtle infiltrates and can sometimes miss early or mild disease manifestations. Positioning and patient movement can also affect image quality.  **CT Scan:**  - **Appearance:** CT scans provide a more detailed and three-dimensional view of the lungs. Pulmonary infiltrates on a CT scan can be characterized by ground-glass opacities (semi-transparent areas indicating partial filling of air spaces), consolidation, nodules, or reticular patterns. CT can differentiate between various types of infiltrates based on their density, distribution, and associated features.  - **Advantages:** CT scans are more sensitive and can detect smaller or more subtle infiltrates that chest X-rays might miss. They also provide better anatomical localization and can help identify underlying causes, such as tumors, infections, or interstitial lung diseases.  **Comparison:** While chest X-rays are valuable for rapid assessment and screening, CT scans offer superior detail and diagnostic accuracy for evaluating pulmonary infiltrates, guiding further management and treatment decisions."
    }
]

initial_instruction = (
    "You are a medical assistant. "
    "Your goal is to answer questions related to medical conditions. "
    "For questions about health except for Atelectasis, Cardiomegaly, Effusion, Infiltration, Mass, Nodule, Pneumonia, Pneumothorax, "
    "Consolidation, Edema, Emphysema, pulmonary edema, lung consolidation, lung nodules, lung cancer, enlarged heart, "
    "you should answer those questions, with the answer add the disclaimer 'Disclaimer: I am only specialized in certain lung and chest diseases.' "
    "For other prompts not related to health, respond with: 'I can only assist with questions related to medical conditions, particularly those concerning lung and chest diseases. "
    "If your inquiry falls outside of health-related topics, I won’t be able to respond. Please keep the questions focused on medical conditions.'"
)

def ask_medical_chatbot(conversation_history):
    parts = [{"text": initial_instruction}]
    
    for exchange in conversation_history:
        parts.append({"text": exchange["user"]})
        if exchange["assistant"]:
            parts.append({"text": exchange["assistant"]})

    payload = {
        "contents": [{
            "parts": parts
        }]
    }

    try:
        response = requests.post(
            url='https://generativelanguage.googleapis.com/v1beta/tunedModels/book2modifiedanswers-6vt4z4xscfbz:generateContent',
            headers=headers,
            json=payload
        )
    except requests.exceptions.RequestException as e:
        return f"Error: Unable to connect to the API. {e}"

    if response.status_code == 200:
        response_json = response.json()
        try:
            # Extract the assistant's reply
            answer = response_json['candidates'][0]['content']['parts'][-1]['text']
            return answer.strip()
        except (KeyError, IndexError) as e:
            return "Error: Unable to parse the response structure."
    else:
        return f"Error: {response.status_code} - {response.text}"

def evaluate_semantics_bertscore(reference, candidate):
    """
    Evaluates semantic similarity between reference and candidate texts using BERTScore.
    
    Returns:
        precision, recall, f1: BERTScore metrics
    """
    P, R, F1 = score([candidate], [reference], lang="en", verbose=False)
    return P[0].item(), R[0].item(), F1[0].item()

def run_automated_tests():
    print("\nRunning Automated Test Cases with BERTScore Evaluation...\n")
    passed = 0
    failed = 0

    for idx, test in enumerate(test_cases, 1):
        query = test["query"]
        expected = test["expected"]

        if not expected:
            print(f"Test Case {idx}: Skipped (No expected answer provided)\n")
            continue

        conversation_history = [{"user": query, "assistant": ""}]
        assistant_reply = ask_medical_chatbot(conversation_history)
        assistant_reply = assistant_reply.replace('*', '')

        precision, recall, f1 = evaluate_semantics_bertscore(expected, assistant_reply)
        threshold_f1 = 0.75  

        if f1 >= threshold_f1:
            result = "PASSED"
            passed += 1
        else:
            result = "FAILED"
            failed += 1

        print(f"Test Case {idx}: {result}")
        print(f"Query: {query}")
        print(f"Expected: {expected}")
        print(f"Assistant: {assistant_reply}")
        print(f"BERTScore - Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}")
        if f1 < threshold_f1:
            print(f"Note: F1 score below threshold of {threshold_f1}")
        print("-" * 80)

    total = passed + failed
    print(f"\nTest Results: {passed} Passed, {failed} Failed out of {total} Test Cases.\n")

def main():
    print("Welcome to the Medical Chatbot specialized in lung and chest conditions.")
    print("You can ask questions about Atelectasis, Cardiomegaly, and other related conditions.")
    print("Type 'exit' or 'quit' to end the session.")
    print("Type 'run_tests' to execute automated test cases.\n")

    conversation_history = []

    while True:
        question = input("You: ").strip()

        if question.lower() in ['exit', 'quit']:
            print("Assistant: Thank you for using the Medical Chatbot. Goodbye!")
            break  

        if question.lower() == 'run_tests':
            run_automated_tests()
            continue

        if not question:
            print("Assistant: Please enter a valid question.\n")
            continue  

        conversation_history.append({"user": question, "assistant": ""})

        assistant_reply = ask_medical_chatbot(conversation_history)

        assistant_reply = assistant_reply.replace('*', '')

        test_case = next((tc for tc in test_cases if tc["query"].lower() == question.lower()), None)
        if test_case:
            expected_answer = test_case["expected"]
            precision, recall, f1 = evaluate_semantics_bertscore(expected_answer, assistant_reply)
            threshold_f1 = 0.75  

            print(f"BERTScore - Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}")
            if f1 < threshold_f1:
                print(f"Assistant: [Semantic Warning] The response may not be semantically accurate or fully aligned with the medical condition specified.\n")
        else:
            print("No semantic evaluation available for this query.\n")

        conversation_history[-1]["assistant"] = assistant_reply

        print(f"Assistant: {assistant_reply}\n")

if __name__ == "__main__":
    main()
